<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/split_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Data From Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%mkdir data
%cd data
%mkdir raw
%cd raw

/content/data
/content/data/raw


In [3]:
!cp /content/drive/MyDrive/amex-default-prediction/data/raw/amex-default-prediction.zip .

In [4]:
!unzip amex-default-prediction.zip train_data.csv

Archive:  amex-default-prediction.zip
  inflating: train_data.csv          


In [5]:
!unzip amex-default-prediction.zip train_labels.csv

Archive:  amex-default-prediction.zip
  inflating: train_labels.csv        


In [6]:
import gc

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:

TRAIN_DATA_PATH = '/content/data/raw/train_data.csv'
TRAIN_LABELS_PATH = '/content/data/raw/train_labels.csv'

In [8]:
def read_cols(cols):
  df = pd.read_csv(TRAIN_DATA_PATH, usecols=cols)
  df = append_label(df)
  return df

In [9]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [10]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458913 entries, 0 to 458912
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_ID  458913 non-null  object
 1   target       458913 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [11]:
def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [12]:
pd.read_csv(TRAIN_DATA_PATH, nrows=10).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 15.0+ KB


In [13]:
columns = pd.read_csv(TRAIN_DATA_PATH, nrows=10).dtypes

In [14]:
cust_id_stmt_date_df = pd.read_csv(TRAIN_DATA_PATH, usecols=['customer_ID', 'S_2'])

In [15]:
cust_id_stmt_date_df.shape

(5531451, 2)

In [16]:
cust_id_stmt_date_df.head()

,customer_ID,S_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16


In [18]:
cust_id_stmt_date_df.iloc[0]['customer_ID']

'0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'

In [24]:
idx = cust_id_stmt_date_df[cust_id_stmt_date_df['customer_ID'] == '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a'].index.tolist()
idx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [25]:
cust_id_stmt_date_df.iloc[14]['customer_ID']

'00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5'

In [31]:
customers = ['00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5', '0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a']
idx = cust_id_stmt_date_df[cust_id_stmt_date_df['customer_ID'].isin(customers)].index.tolist()
idx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [32]:
df = pd.read_csv(TRAIN_DATA_PATH, skiprows=(lambda x: x not in idx))

In [ ]:
df

In [41]:
customers = cust_id_stmt_date_df.customer_ID.unique().tolist()

In [42]:
len(customers)

458913

In [44]:
customers[0:10]

['0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a',
 '00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5',
 '00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1',
 '000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc',
 '00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed',
 '000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370150fc2f8bdca6202c',
 '000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4',
 '0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381',
 '0000f99513770170a1aba690daeeb8a96da4a39f11fc27da5c30a79db61c1e85',
 '00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e4063bdf6b0f7d51eb6']

In [57]:
idx = cust_id_stmt_date_df[cust_id_stmt_date_df['customer_ID'].isin(customers[0:2])].index.tolist()
len(idx)

26

In [58]:
df = pd.read_csv(TRAIN_DATA_PATH, skiprows=(lambda x: x not in idx))

In [64]:
import dask
import dask.dataframe as dd


In [ ]:
!pip install dask[dataframe]

In [66]:
df = dd.read_csv(TRAIN_DATA_PATH)

In [94]:
ddf1 = df.partitions[0].compute()

In [ ]:
ddf1[ddf1.customer_ID.isin(customers[0:10])]

In [108]:
rdf = pd.DataFrame()
rdf

""


In [116]:

for i in range(df.npartitions):
  print(i)
  ddf1 = df.partitions[i].compute()
  rdf = rdf.append(ddf1[ddf1.customer_ID.isin(customers[0:100000])], ignore_index=True)
  print(rdf.shape)

0
(142224, 190)
1
(163822, 190)
2
(185409, 190)
3
(206976, 190)
4
(228603, 190)
5
(250189, 190)
6
(271773, 190)
7
(293358, 190)
8
(314960, 190)
9
(336552, 190)
10
(358164, 190)
11
(379778, 190)
12
(401381, 190)
13
(422978, 190)
14
(444560, 190)
15
(466169, 190)
16
(487772, 190)
17
(509355, 190)
18
(530938, 190)
19
(552535, 190)
20
(574104, 190)
21
(595680, 190)
22
(617273, 190)
23
(638872, 190)
24
(660466, 190)
25
(682074, 190)
26
(703676, 190)
27
(725266, 190)
28
(746861, 190)
29
(768475, 190)
30
(790075, 190)
31
(811653, 190)
32
(833227, 190)
33
(854827, 190)
34
(876415, 190)
35
(898023, 190)
36
(919613, 190)
37
(941212, 190)
38
(962807, 190)
39
(984395, 190)
40
(1005965, 190)
41
(1027536, 190)
42
(1049150, 190)
43
(1070755, 190)
44
(1092342, 190)
45
(1113931, 190)
46
(1135542, 190)
47
(1157143, 190)
48
(1178710, 190)
49
(1200289, 190)
50
(1221864, 190)
51
(1243447, 190)
52
(1265030, 190)
53
(1286654, 190)
54
(1308239, 190)
55
(1326261, 190)
56
(1326261, 190)
57
(1326261, 190)
58
(13

In [118]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326261 entries, 0 to 1326260
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 1.9+ GB


In [120]:
rdf.to_parquet(path='data_1.parquet')

In [121]:
!cp data_1.parquet /content/drive/MyDrive/amex-default-prediction/data/raw/

In [128]:
rdf = pd.DataFrame()
for i in range(df.npartitions):
  print(i)
  ddf1 = df.partitions[i].compute()
  ddf1['S_2'] = pd.to_datetime(ddf1["S_2"])
  rdf = rdf.append(ddf1[ddf1.S_2.dt.to_period('M')== '2017-05'], ignore_index=True)
  print(rdf.shape)

0
(1560, 190)
1
(3126, 190)
2
(4688, 190)
3
(6240, 190)
4
(7798, 190)
5
(9352, 190)
6
(10904, 190)
7
(12464, 190)
8
(14012, 190)
9
(15568, 190)
10
(17148, 190)
11
(18703, 190)
12
(20266, 190)
13
(21818, 190)
14
(23379, 190)
15
(24937, 190)
16
(26494, 190)
17
(28054, 190)
18
(29612, 190)
19
(31173, 190)
20
(32750, 190)
21
(34312, 190)
22
(35861, 190)
23
(37418, 190)
24
(38975, 190)
25
(40527, 190)
26
(42083, 190)
27
(43634, 190)
28
(45195, 190)
29
(46751, 190)
30
(48316, 190)
31
(49879, 190)
32
(51444, 190)
33
(53012, 190)
34
(54589, 190)
35
(56159, 190)
36
(57724, 190)
37
(59289, 190)
38
(60858, 190)
39
(62428, 190)
40
(64000, 190)
41
(65564, 190)
42
(67119, 190)
43
(68671, 190)
44
(70235, 190)
45
(71783, 190)
46
(73361, 190)
47
(74916, 190)
48
(76479, 190)
49
(78056, 190)
50
(79620, 190)
51
(81180, 190)
52
(82761, 190)
53
(84321, 190)
54
(85888, 190)
55
(87448, 190)
56
(89017, 190)
57
(90597, 190)
58
(92157, 190)
59
(93734, 190)
60
(95310, 190)
61
(96873, 190)
62
(98418, 190)
63
(9997

In [129]:
rdf.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2017-05-07,0.922563,0.567403,0.033713,1.007497,0.005594,0.094655,0.003162,0.010466,...,NaN,NaN,NaN,0.007742,0.002693,0.009851,NaN,0.000996,0.003254,0.004469
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-05-12,0.884522,0.003399,0.003495,0.818569,0.007541,NaN,0.007537,0.000859,...,NaN,NaN,NaN,0.003084,0.000304,0.000891,NaN,0.002980,0.009026,0.002959
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2017-05-29,0.581232,0.206867,0.279991,1.004374,0.001605,0.149216,0.008668,0.008219,...,NaN,NaN,NaN,0.004132,0.003133,0.007118,NaN,0.003069,0.000492,0.004728
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,...,NaN,NaN,NaN,0.008103,0.006132,0.003267,NaN,0.008814,0.001211,0.001624


In [130]:
rdf.S_2.unique()

array(['2017-05-28T00:00:00.000000000', '2017-05-07T00:00:00.000000000',
       '2017-05-12T00:00:00.000000000', '2017-05-29T00:00:00.000000000',
       '2017-05-30T00:00:00.000000000', '2017-05-19T00:00:00.000000000',
       '2017-05-23T00:00:00.000000000', '2017-05-15T00:00:00.000000000',
       '2017-05-17T00:00:00.000000000', '2017-05-13T00:00:00.000000000',
       '2017-05-11T00:00:00.000000000', '2017-05-14T00:00:00.000000000',
       '2017-05-18T00:00:00.000000000', '2017-05-16T00:00:00.000000000',
       '2017-05-26T00:00:00.000000000', '2017-05-21T00:00:00.000000000',
       '2017-05-31T00:00:00.000000000', '2017-05-27T00:00:00.000000000',
       '2017-05-08T00:00:00.000000000', '2017-05-09T00:00:00.000000000',
       '2017-05-24T00:00:00.000000000', '2017-05-06T00:00:00.000000000',
       '2017-05-04T00:00:00.000000000', '2017-05-05T00:00:00.000000000',
       '2017-05-10T00:00:00.000000000', '2017-05-03T00:00:00.000000000',
       '2017-05-22T00:00:00.000000000', '2017-05-01